In [ ]:
import torch
import transformers
from torch import cuda, bfloat16


device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
device

In [ ]:
model_name = 'REELICIT/zephyr-7b-beta-ReqBrain'

model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, use_fast = False)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model.eval()
model.to(device)
print(f"Model loaded on {device}")

In [ ]:
stop_token_ids = [
    tokenizer.convert_tokens_to_ids(x) for x in [[tokenizer.eos_token], ["<", "|", "user"]]
]

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [ ]:
pipe = transformers.pipeline(
    model = model,
    tokenizer = tokenizer,
    return_full_text = True, # Set it to True when combining with LangChain
    task = 'text-generation',
    device = device,
    stopping_criteria=stopping_criteria,  
    temperature = 0.2,
    top_p = 0.15,  
    top_k=0,  
    max_new_tokens = 512,  
    repetition_penalty = 1.3
)

In [ ]:
import datasets

evaluation_set = datasets.load_from_disk("./models_prediction_dataset")
evaluation_set

In [ ]:
from tqdm import tqdm

completion = []

instruction = "You are a professional requirements engineer who helps users brainstorm more software requirements."

for query in tqdm(evaluation_set['query']):
    result = pipe(f"<|system|>\n{instruction}\n<|user|>\n{query}\n<|assistant|>\n")
    result = result[0]['generated_text'].split('\n<|assistant|>\n')[-1].strip("<|user")
    result = result.strip(" ")
    completion.append(result)

In [ ]:
evaluation_set = evaluation_set.add_column("zephyr_7b_beta_preds", completion)

In [ ]:
evaluation_set.save_to_disk("./models_prediction_dataset")